# ${\huge \Lambda}$
*Nothing but Master*

## Introduzione
### SSA
SSA (*Static Single Assignment*) è una forma di rappresentazione del codice in cui ogni variabile viene assegnata **una** sola volta. Quindi, consideriamo il codice
```
_Bool x;
int y;

y = 8;
if(x)
    y--;
else
    y++;

assert(y == 7 || y == 9);
```

tale codice diventa
```
_Bool x;
int y0;
int y1;
int y2;
int y3;


y0 = 8;
if(x)
    y1 = y0 - 1;
else
    y2 = y0 + 1;
    
y3 = x ? y1 : y2;
assert(y3 == 7 || y3 == 9);
```

Questa metodologia, popolare tra i compilatori, di fatto tratta le variabili come se fossero delle costanti. Lo scopo è quello di tradurre un programma imperativo in un'equazione. Di fatto, otterremmo qualcosa di concettualmente molto simile anche se provassimo a tradurre lo stesso codice in un programma funzionale. a questo punto possiamo scrivere il nostro programma come l'equazione che segue

\begin{equation}
y_0=8 \wedge (y_1=y_0-1) \wedge (y_2=y_0+1) \wedge (y_0=\mathrm{true}\rightarrow y_3=y_1) \wedge (y_0=\mathrm{false}\rightarrow y_3=y_2) \wedge \\ \wedge (y_3=7 \vee y_3=9)
\end{equation}

che, se vogliamo che il programma rispetti la specifica che abbiamo dato (in questo caso con l'assert), deve risultare **valida** (cioè tautologica - vera in tutti i casi).

Il file in SMT-LIB2 con questo esempio si trova in `Lambda/doc/examples/sample1.smt2`.
Qui si da una dimostrazione dello stesso codice scritto però in Python (utilizzando i binding di Z3)

In [12]:
from z3 import *

s = Solver()

y1, y2, y3, y4 = Ints("y1 y2 y3 y4")
x0 = Bool("x0")

# Precondizione (inizializzazione)
s.add(y1 == 8)

# Corpo del programma
s.add(y2 == y1 -1, y3 == y1 + 1, Implies(x0 == True,y4 == y2), Implies(x0 == False, y4 == y3))

# Postcondizione (negata)
s.add(Not(Or(y4 == 7, y4 == 9)))

# Check di soddisfacibilità (deve risultare unsat)
s.check()

unsat

## Esempio
Consideriamo un esempio un po' più complesso:
```
#define DIM 5

int x;
int A[DIM];
int i = 0;
int found = 0; 

while(i < DIM) {
    if(x == A[i]) 
        found = 1;
    i++; 
}
```
dove la costante `DIM` rappresenta un numero arbitrario. Vogliamo verificare la seguente specifica:

\begin{equation}
\mathrm{found}=1\longleftrightarrow x\in A
\end{equation}

Sulla base di quanto detto nell'Introduzione, iniziamo a creare la formula di inizializzazione:

\begin{equation}
i_0=0 \wedge f_0=0
\end{equation}

per il corpo del ciclo (formula che andrà poi ripetuta `DIM` volte):

\begin{equation}
f_1 = 1 \wedge f_2=f_0 \wedge (x=A(i_0) \rightarrow f_3=f_1) \wedge (x\neq A(i_0) \rightarrow f_3=f_2) \wedge (i_1 = i_0+1)
\end{equation}

Mentre la postcondizione diventa:

\begin{equation}
f_n=1 \leftrightarrow x = A(i_{n-1})
\end{equation}

dove $n = \mathrm{DIM}$

Anche in questo caso il file in SMT-LIB2 con questo esempio si trova in `Lambda/doc/examples/sample2.smt2`.
Qui si da una dimostrazione dello stesso codice scritto però in Python (utilizzando i binding di Z3)

In [14]:
from z3 import *

def n_found(d) : return (d * 3) + 1

def n_i(d) : return d + 1

def body(f_0, f_1, f_2, f_3, i_0, i_1, _A, _x) :
    return And(f_1 == 1, f_2 == f_0, \
               Implies(_x == Select(_A,i_0), f_3 == f_1), \
               Implies(Not(_x == Select(_A,i_0)),f_3 == f_2), \
               i_1 == (i_0 + 1))

s = Solver()

DIM = int(input("numero di iterazioni: "))
A = Array("A",IntSort(),IntSort())
x = Int("x")
i = [Int(f"i{x}") for x in range(n_i(DIM))]
f = [Int(f"f{x}") for x in range(n_found(DIM))]

init = [And(i0 == 0, f0 == 0)]

tran = []

post = 

for x in range(DIM) :
    if x < (DIM * 3) / 2 :
        tran.append(body(f[0],f[1],f[2],f[3],i[0],i[1],A,x))
        for d in range(3) :
            del f[0]
        del i[0]

numero di iterazioni: 3
[f0, f1, f2, f3, f4, f5, f6, f7, f8, f9]
[i0, i1, i2, i3]
[f3, f4, f5, f6, f7, f8, f9]
[i1, i2, i3]
[f6, f7, f8, f9]
[i2, i3]
[f9]
[i3]
